In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

### Inspecting Train Data

In [94]:
df = pd.read_csv(r'data\video_001.csv')
df.head()

,leftShoulder_x,leftShoulder_y,rightShoulder_x,rightShoulder_y,leftElbow_x,leftElbow_y,rightElbow_x,rightElbow_y,leftWrist_x,leftWrist_y,rightWrist_x,rightWrist_y,label
0,392,152,322,146,402,210,268,160,401,267,230,148,1
1,393,152,319,147,399,210,260,164,402,272,219,147,1
2,391,151,321,147,403,207,269,162,408,270,356,272,1
3,391,150,321,145,403,207,265,166,397,267,335,266,1
4,390,150,322,149,403,210,270,159,406,269,224,148,1


In [95]:
df.describe()

,leftShoulder_x,leftShoulder_y,rightShoulder_x,rightShoulder_y,leftElbow_x,leftElbow_y,rightElbow_x,rightElbow_y,leftWrist_x,leftWrist_y,rightWrist_x,rightWrist_y,label
count,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000
mean,379.285714,147.746032,309.888889,147.349206,402.412698,184.238095,282.222222,177.222222,405.190476,226.682540,277.634921,205.222222,2.619048
std,5.592127,5.665311,4.701102,6.481136,15.449851,41.403488,24.600973,45.937350,37.441937,76.862849,41.038653,82.635118,1.249885
min,367.000000,130.000000,298.000000,129.000000,375.000000,88.000000,245.000000,83.000000,373.000000,19.000000,186.000000,13.000000,1.000000
25%,377.000000,148.000000,308.000000,146.000000,392.000000,156.000000,265.000000,156.000000,387.000000,253.000000,251.500000,141.000000,1.000000
50%,379.000000,150.000000,309.000000,150.000000,397.000000,209.000000,291.000000,210.000000,392.000000,263.000000,290.000000,263.000000,3.000000
75%,382.000000,151.000000,311.500000,152.000000,410.000000,211.000000,295.000000,215.000000,402.500000,267.000000,298.000000,268.000000,4.000000
max,393.000000,154.000000,322.000000,154.000000,446.000000,217.000000,414.000000,219.000000,523.000000,273.000000,404.000000,274.000000,4.000000


In [96]:
df.label.unique()

array([1, 3, 4, 2], dtype=int64)

### Loading Data and Training Model

In [3]:
import numpy as np

In [4]:
data_csv = np.genfromtxt('data/video_001.csv', delimiter=',')

train_x = data_csv[1:50,0:12] / 600
train_y = data_csv[1:50,12:13]
test_x = data_csv[50:,0:12] / 600
test_y = data_csv[50:,12:13]

In [5]:
train_x.shape

(49, 12)

In [6]:
from keras.utils import to_categorical
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

Using TensorFlow backend.


In [7]:
print(train_x[4])
print(train_y[4])

[0.65       0.25       0.53666667 0.24833333 0.67166667 0.35
 0.45       0.265      0.67666667 0.44833333 0.37333333 0.24666667]
[0. 1. 0. 0. 0.]


In [8]:
print(test_x[4])
print(test_y[4])

[0.63166667 0.24666667 0.51333333 0.25166667 0.655      0.355
 0.49333333 0.35333333 0.65333333 0.445      0.49333333 0.45166667]
[0. 0. 0. 0. 1.]


In [9]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(
    10, 
    activation="relu", 
    input_shape=(12, )))
model.add(layers.Dense(8, activation="relu"))
model.add(layers.Dense(5, activation="softmax"))

model.summary()                                         

from keras import optimizers, losses, metrics
model.compile(
    optimizer=optimizers.RMSprop(lr=0.001),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.categorical_accuracy]
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 45        
Total params: 263
Trainable params: 263
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(
    train_x,
    train_y,
    epochs=3,
    batch_size=10
)

Epoch 1/3
49/49 [==============================] - 0s 9ms/step - loss: 1.5298 - categorical_accuracy: 0.3673
Epoch 2/3
49/49 [==============================] - 0s 245us/step - loss: 1.5094 - categorical_accuracy: 0.3878
Epoch 3/3
49/49 [==============================] - 0s 326us/step - loss: 1.4985 - categorical_accuracy: 0.3673


In [11]:
test_loss, test_acc = model.evaluate(test_x, test_y)
print("Loss / Accuracy Evaluation")
print("--------------------------")
print("Loss:     " + str(round(test_loss,5)))
print("Accuracy: " + str(round(test_acc,2)))

14/14 [==============================] - 0s 4ms/step
Loss / Accuracy Evaluation
--------------------------
Loss:     1.39607
Accuracy: 0.79


In [12]:
test_y_pred = model.predict(test_x)

In [13]:
test_y_pred

array([[0.12138621, 0.2043669 , 0.21782994, 0.19924143, 0.2571755 ],
       [0.12083072, 0.20257555, 0.21766469, 0.19971877, 0.25921032],
       [0.12045828, 0.20135248, 0.21687013, 0.19998716, 0.26133195],
       [0.12029037, 0.2011014 , 0.21720842, 0.20025946, 0.26114023],
       [0.12031044, 0.20099978, 0.21689826, 0.20010126, 0.2616903 ],
       [0.11971876, 0.20038189, 0.2170972 , 0.20027792, 0.26252425],
       [0.12013637, 0.20147209, 0.21715093, 0.19994609, 0.2612945 ],
       [0.12025196, 0.20170231, 0.2170756 , 0.19985394, 0.2611162 ],
       [0.11998167, 0.2006515 , 0.21720871, 0.20022787, 0.26193026],
       [0.11992314, 0.20065507, 0.21737687, 0.20030275, 0.26174217],
       [0.12044746, 0.20102696, 0.21679291, 0.20008206, 0.26165062],
       [0.1207404 , 0.2014052 , 0.21647029, 0.19996926, 0.2614149 ],
       [0.12136436, 0.20162149, 0.21646275, 0.19997257, 0.2605789 ],
       [0.12136436, 0.20162149, 0.21646273, 0.19997257, 0.2605789 ]],
      dtype=float32)

In [14]:
model.save('model.hdf5')

In [16]:
import tensorflowjs as tfjs

In [17]:
tfjs.converters.save_keras_model(model, 'model_tfjs')